In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from statsmodels.base.model import GenericLikelihoodModel
from scipy import stats #for kernel function

In [2]:
#TODO:

#debug value function
#debug kappa function

#states vs X

#implement class using statsmodles
#iterate AM style


#paramterize cost function (for different specs)

The code below references the following two sources.

John Rust's website:
https://editorialexpress.com/jrust/nfxp.html

Victor Aguirregabiria and Pedro Mira's website:
http://individual.utoronto.ca/vaguirre/wpapers/program_code_survey_joe_2008.html

Victor Aguirregabiria and Pedro Mira's 2002 paper
https://www.jstor.org/stable/3082006


In [17]:
#fix the bus .dat because Aguirregabiria and Mira hate everyone
data = np.fromfile('bus1234.dat')
data = data.reshape(len(data)/6,6)
data = pd.DataFrame(data,columns=['id','group','year','month','replace','miles'])

#save to .csv so other people don't need to be confused
data.to_csv("bus1234.csv")

#divide by 1e6
data['miles'] = (data['miles'])/1e6

#switch to date time for ease 
data['date'] = pd.to_datetime(data[['year', 'month']].assign(Day=1))
data = data[['id','group','date','replace','miles']]

#lag date
date_lag = data.copy()
date_lag['date'] = date_lag['date'] - pd.DateOffset(months=1)
data = data.merge(date_lag, how='left', on=['id','group','date'] , suffixes=('','_next'))
data = data.dropna()

print data.max()
print data.min()

id                              162
group                        530875
date            1985-04-01 00:00:00
replace                           1
miles                      0.388254
replace_next                      1
miles_next                 0.388254
dtype: object
id                               59
group                            50
date            1975-09-01 00:00:00
replace                           0
miles                             0
replace_next                      0
miles_next                        0
dtype: object


In [4]:
#constants
BETA = .9999
GAMMA = .5772 #euler's constant

#size of step in discretization
STEP = .002
#STEP = 3000

In [5]:
def miles_pdf(i_obs, x_obs, x_next):
    """estimation of mileage pdf following AM using the
    kernel function
    
    this corresponds to pdfdx in AM's code"""
    
    #figure out max number of steps
    dx = (1-i_obs)*(x_next - x_obs) + i_obs*x_next
    
    #number of states
    dx_states = np.arange(dx.min(),dx.max() +STEP , STEP)
    
    #use kernel groups to make pdf
    kernel1 = stats.gaussian_kde(dx, bw_method='silverman')
    pdfdx = kernel1(dx_states)
    
    return np.array([pdfdx/pdfdx.sum()]).transpose()


MILES_PDF = miles_pdf(data['replace'], data['miles'], data['miles_next'])

In [6]:
def transition_1(i_obs, x_obs , x_next):
    """calculate transitions probabilities,
    non-parametrically
    
    this corresponds to fmat2 in AM's code"""
    
    #transitions when i=1
    num_states = ( x_obs.max()/STEP).astype(int) + 2
    x = np.arange(x_obs.min(),x_obs.max() + STEP, STEP)  
    
    pdfdx = miles_pdf(i_obs, x_obs, x_next).transpose()
    
    #zero probability of transitioning to large states
    zeros = np.zeros( (num_states,num_states-pdfdx.shape[1]) )
    
    #transitioning to first state and 'jumping' dx states
    fmat1 = np.tile(pdfdx,(num_states,1))
    fmat1 = np.concatenate( (fmat1, zeros), axis=1 )

    return fmat1

FMAT1 = transition_1(data['replace'], data['miles'],data['miles_next'])

In [7]:
def transition_0(i_obs, x_obs , x_next):
    """calculate transitions probabilities,
    non-parametrically
    
    this corresponds to fmat1 in AM's code"""
    
    num_states = (x_obs.max()/STEP).astype(int) + 2
    pdfdx = miles_pdf(i_obs, x_obs, x_next).transpose()
    
    #initialize fmat array, transitions when i=0
    end_zeros = np.zeros((1,num_states -pdfdx.shape[1]))
    fmat0 = np.concatenate( (pdfdx,end_zeros), axis=1 )

    for row in range(1,num_states):
        
        #this corresponds to colz i think
        cutoff = (num_states - row - pdfdx.shape[1] )
        
        #case 1 far enough from the 'end' of the matrix
        if cutoff >= 0:
            start_zeros = np.zeros((1,row))
            end_zeros = np.zeros((1,num_states - pdfdx.shape[1] - row))
            fmat_new = np.concatenate( (start_zeros,pdfdx,end_zeros), axis=1 )
            fmat0 = np.concatenate((fmat0,fmat_new))
       
        #case 2, too far from the end and need to adjust probs
        else:
            pdf_adj = pdfdx[:,0:cutoff]
            pdf_adj = pdf_adj/pdf_adj.sum(axis=1)
            
            start_zeros = np.zeros((1,row))
            fmat_new = np.concatenate( (start_zeros,pdf_adj), axis=1 )
            fmat0 = np.concatenate((fmat0,fmat_new))
            
    return fmat0

FMAT0 = transition_0(data['replace'],data['miles'],data['miles_next'])

PR_TRANS = FMAT0, FMAT1

In [8]:
def initial_pr(i_obs, x_obs, d=0):
    """initial the probability of view a given state following AM.
    Seems like it just involves logit to predict
    
    Third arguement involves display"""
    
    X = np.array([x_obs, x_obs**2, x_obs**3]).transpose()
    X = sm.add_constant(X)
    
    model = sm.Logit(i_obs,X)
    fit = model.fit(disp=d)
    if d: print fit.summary()
    
    states = np.arange(x_obs.min(),x_obs.max() +STEP , STEP)
    
    states = np.array([states, states**2, states**3]).transpose()
    states = sm.add_constant(states)
    
    return fit.predict(states)

PR_OBS = initial_pr(data['replace'], data['miles'], d=1)

Optimization terminated successfully.
         Current function value: 0.036201
         Iterations 23
                           Logit Regression Results                           
Dep. Variable:                replace   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8152
Method:                           MLE   Df Model:                            3
Date:                Tue, 15 Jan 2019   Pseudo R-squ.:                  0.1671
Time:                        10:48:22   Log-Likelihood:                -295.26
converged:                       True   LL-Null:                       -354.51
                                        LLR p-value:                 1.623e-25
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -17.3138      4.188     -4.134      0.000     -25.522      -9.105
x1           149.3100     56

In [9]:
def hm_value(theta1, cost, i_obs, x_obs, pr_obs, pr_trans):
    """calculate value function using hotz miller approach"""
    
    #set up matrices, transition is deterministic
    trans0, trans1 = pr_trans
    
    #should probably make these class parameters
    num_states = ( x_obs.max()/STEP).astype(int) + 2
    x = np.arange(x_obs.min(),x_obs.max() + STEP, STEP) 
    
    
    #calculate value function for all state
    pr_tile = np.tile( pr_obs.reshape(num_states,1), (1,num_states))
    
    denom = (np.identity(num_states) - BETA*(1-pr_tile)*trans0 - BETA*pr_tile*trans1)
    
    numer = ( (1-pr_obs)*(theta1*x  + GAMMA - np.log(1-pr_obs)) + 
                 pr_obs*(cost+ GAMMA - np.log(pr_obs) ) )
    
    value = np.linalg.inv(denom).dot(numer)
    return value


VALUE = hm_value(.6, -10, data['replace'], data['miles'], PR_OBS, PR_TRANS)

In [10]:
print VALUE

[6000.88568503 6000.90075705 6000.91505934 6000.92859186 6000.94135452
 6000.95334727 6000.96457003 6000.97502271 6000.98470525 6000.99361754
 6001.00175949 6001.009131   6001.01573196 6001.02156223 6001.0266217
 6001.03091021 6001.03442761 6001.03717375 6001.03914844 6001.0403515
 6001.04078274 6001.04044197 6001.03932899 6001.03744361 6001.03478566
 6001.03135498 6001.02715144 6001.02217498 6001.01642558 6001.00990333
 6001.00260839 6000.9945411  6000.98570195 6000.97609162 6000.96571108
 6000.95456155 6000.94264462 6000.92996229 6000.91651701 6000.90231176
 6000.88735015 6000.87163645 6000.85517568 6000.83797372 6000.82003737
 6000.80137443 6000.78199382 6000.76190562 6000.74112117 6000.71965317
 6000.69751571 6000.67472437 6000.65129627 6000.62725008 6000.60260614
 6000.57738637 6000.55161439 6000.52531543 6000.49851636 6000.47124563
 6000.44353321 6000.41541053 6000.38691037 6000.35806677 6000.32891492
 6000.29949096 6000.26983189 6000.23997535 6000.2099595  6000.17982273
 6000.14

In [12]:
def hm_prob(theta1, cost, i_obs, x_obs, pr_obs, pr_trans):
    """calculate kappa using value function"""
    
    value = hm_value(theta1, cost, i_obs, x_obs, pr_obs, pr_trans)
    value = value - value.min() # subtract out smallest value
    trans0, trans1 = pr_trans
    
    #should probably make these class parameters
    num_states = ( x_obs.max()/STEP).astype(int) + 2
    x = np.arange(x_obs.min(),x_obs.max() + STEP, STEP) 
    
    delta1 = np.exp( cost + BETA*trans1.dot(value))
    delta0 = np.exp( x*theta1 + BETA*trans0.dot(value) )
    
    return delta1/(delta1+delta0)


hm_prob(-.6, -10, data['replace'], data['miles'], PR_OBS, PR_TRANS)

array([4.53978959e-05, 4.99552866e-05, 5.49281558e-05, 6.03498764e-05,
       6.62560484e-05, 7.26846040e-05, 7.96759142e-05, 8.72728965e-05,
       9.55211220e-05, 1.04468923e-04, 1.14167499e-04, 1.24671024e-04,
       1.36036747e-04, 1.48325095e-04, 1.61599772e-04, 1.75927854e-04,
       1.91379877e-04, 2.08029924e-04, 2.25955707e-04, 2.45238640e-04,
       2.65963905e-04, 2.88220514e-04, 3.12101362e-04, 3.37703271e-04,
       3.65127024e-04, 3.94477394e-04, 4.25863158e-04, 4.59397109e-04,
       4.95196046e-04, 5.33380771e-04, 5.74076060e-04, 6.17410630e-04,
       6.63517100e-04, 7.12531940e-04, 7.64595402e-04, 8.19851455e-04,
       8.78447702e-04, 9.40535289e-04, 1.00626881e-03, 1.07580618e-03,
       1.14930856e-03, 1.22694017e-03, 1.30886822e-03, 1.39526271e-03,
       1.48629629e-03, 1.58214410e-03, 1.68298359e-03, 1.78899433e-03,
       1.90035780e-03, 2.01725717e-03, 2.13987707e-03, 2.26840336e-03,
       2.40302284e-03, 2.54392296e-03, 2.69129158e-03, 2.84531659e-03,
      

In [19]:
from scipy.optimize import minimize

class HotzMiller():
    """class for estimating the values of R and theta
    using the Hotz Miller routine and the helper functions
    above"""
    
    def __init__(self, i, x, x_next):
        
        #transitions
        self.pr_obs = initial_pr(i, x)
        self.trans =  transition_0(i,x,x_next), transition_1(i,x,x_next)
        
        #should probably make these class parameters
        self.num_states = ( x.max()/STEP).astype(int) + 2
        self.states = np.arange(x.min(),x.max() + STEP, STEP)
        
        #data
        self.x = x
        self.x_next = x_next
        self.i = i
        
        #parameters
        self.theta1 = 0
        self.R = 0
        
        
    def likelihood(self, params): 
        theta1, R = params
        
        # Input our data into the model
        i = self.i
        x = (self.x/STEP).astype(int)*STEP #discretized x
        
        
        #set up hm state pr
        prob = hm_prob(theta1, R, self.i, self.x, self.pr_obs, self.trans).transpose()
        prob = interp1d(self.states, prob)
       
        #ensure only positive numbers
        prob = prob(x)
        prob = np.maximum(.0000001, prob)
        prob = np.minimum(.9999999, prob)
        
        log_likelihood = (1-i)*np.log(1-prob) + i*np.log(prob)
        
        return -log_likelihood.sum()
    
    
    def fit(self):
        result = minimize(self.likelihood, [-5,-10], method = 'Nelder-Mead', options={'disp': False})
        self.theta1, self.R = result.x

        
model_hm = HotzMiller(data['replace'], data['miles'],data['miles_next'])
model_hm.fit()


print '\n theta_1:%s, R:%s'%(round(model_hm.theta1,4) , round(model_hm.R,4))


 theta_1:-0.5218, R:-10.1309
